In [ ]:
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
import time

def html_to_jpg(
    html_path: str,
    output_jpg: str,
    width: int = 1280,
    height: int = 720,
    wait_seconds: float = 1.5,
    jpeg_quality: int = 85,
    full_page: bool = True
):
    """
    Render an HTML file to a JPG image using headless Chrome.

    Args:
        html_path: Path to the input HTML file.
        output_jpg: Path to the output JPG file.
        width: Browser viewport width in pixels.
        height: Browser viewport height in pixels.
        wait_seconds: Time to wait for content to settle (JS, fonts, maps).
        jpeg_quality: JPEG quality (1-95).
        full_page: If True, screenshot the full page height; else viewport only.

    Returns:
        Path to the written JPG.
    """
    html_path = Path(html_path).resolve()
    output_jpg = Path(output_jpg).resolve()
    output_png = output_jpg.with_suffix(".png")  # temporary PNG

    if not html_path.exists():
        raise FileNotFoundError(f"HTML not found: {html_path}")

    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--hide-scrollbars")
    chrome_options.add_argument(f"--window-size={width},{height}")
    chrome_options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                              options=chrome_options)
    try:
        driver.get(f"file:///{html_path.as_posix()}")
        time.sleep(wait_seconds)

        if full_page:
            # Determine full page height via JS and resize
            total_height = driver.execute_script(
                "return Math.max(document.body.scrollHeight,"
                "document.documentElement.scrollHeight);"
            )
            # Cap to avoid extremely tall pages
            max_height = max(height, min(total_height, 10000))
            driver.set_window_size(width, max_height)
            time.sleep(0.5)

        driver.save_screenshot(str(output_png))

        # Convert PNG → JPG with specified quality
        with Image.open(output_png) as im:
            # If PNG has alpha, flatten onto white background
            if im.mode in ("RGBA", "LA"):
                bg = Image.new("RGB", im.size, (255, 255, 255))
                bg.paste(im, mask=im.split()[-1])
                im = bg
            else:
                im = im.convert("RGB")

            im.save(output_jpg, "JPEG", quality=jpeg_quality, optimize=True)

        # Cleanup temp PNG
        try:
            output_png.unlink(missing_ok=True)
        except Exception:
            pass

        print(f"Saved JPG: {output_jpg}")
        return output_jpg
    finally:
        driver.quit()

# Example usage:
html_to_jpg("outputs/polygon_combined.html", "outputs/polygon_combined.jpg",
            width=1200, height=850, wait_seconds=2, jpeg_quality=95, full_page=True)

Saved JPG: C:\Users\jonas\Desktop\Tu-Wien\Semester_4\Project\RemoteSensing-InterdisciplinaryProject\dynamic-flood-visualization\visualizations\jonas\outputs\polygon_combined.jpg


WindowsPath('C:/Users/jonas/Desktop/Tu-Wien/Semester_4/Project/RemoteSensing-InterdisciplinaryProject/dynamic-flood-visualization/visualizations/jonas/outputs/polygon_combined.jpg')